In [48]:
import numpy as np
import pandas as pd
import calendar
import json
import logging
from datetime import timedelta, datetime, date

# Setup logging: adjust the level as needed (DEBUG, INFO, WARNING, etc.)
logging.basicConfig(level=logging.INFO,
                    format='%(asctime)s - %(levelname)s - %(message)s')


In [49]:
cols = ['PERMNO', 'start', 'ending', 'GVKEY', 'TIC', 'CUSIP', 'COMPANYID', 'LINKDT', 'LINKENDDT']
sp500_metadata_df = pd.read_csv("./data/metadata/sp500_list_w_gvkey_companyid.csv")[cols]
sp500_metadata_df['GVKEY'] = sp500_metadata_df['GVKEY'].astype(str).str.zfill(6)
sp500_metadata_df.drop_duplicates()

,PERMNO,start,ending,GVKEY,TIC,CUSIP,COMPANYID,LINKDT,LINKENDDT
0,10006,1957-03-01,1984-07-18,001010,4165A,00099V004,3611396,1962-01-31,1984-06-28
1,10006,1957-03-01,1984-07-18,001010,4165A,00099V004,3611396,1950-05-01,1962-01-30
2,10030,1957-03-01,1969-01-08,001057,8892A,001935006,58214981,1964-01-01,1967-01-30
3,10030,1957-03-01,1969-01-08,001057,8892A,001935006,58214981,1967-01-31,1967-12-29
4,10057,1957-03-01,1992-07-02,001098,AMT.1,004626107,247705,1968-01-01,1996-07-02
...,...,...,...,...,...,...,...,...,...
2117,93159,2012-07-31,2016-03-29,002270,VAL,G9460G101,269488,2012-05-22,2020-08-17
2118,93246,2021-03-22,2023-12-29,183736,GNRC,368736104,78749412,2010-02-11,NaN
2119,93422,2010-07-01,2015-06-30,154357,QEP,74733V100,1565109,2010-07-01,2021-03-31
2120,93429,2017-03-01,2023-12-29,184500,CBOE,12503M108,32624297,2010-06-15,NaN


In [50]:
monthly_metadata_df = pd.read_sas("./data/msfepssurprise.sas7bdat")
monthly_metadata_df['TICKER'] = monthly_metadata_df.TICKER.apply(lambda x: x.decode('utf-8') if isinstance(x, bytes) else x)
monthly_metadata_df['PERMNO'] = monthly_metadata_df.PERMNO.apply(lambda x: x.decode('utf-8') if isinstance(x, bytes) else x).astype(int)
monthly_metadata_df['CUSIP'] = monthly_metadata_df.CUSIP.apply(lambda x: x.decode('utf-8') if isinstance(x, bytes) else x)
monthly_metadata_df

,PERMNO,DATE,CUSIP,NCUSIP,COMNAM,TICKER,PERMCO,SHRCD,SHRCLS,ISSUNO,...,MMCNT,NSDINX,VWRETD,VWRETX,EWRETD,EWRETX,SPRTRN,cap,capbillion,tradevolmillion
0,10032,2000-08-31,72913210,b'72913210',b'PLEXUS CORP',PLXS,7980.0,11.0,NaN,10437.0,...,31.0,2.0,0.074984,0.073803,0.059300,0.057971,0.060699,2.848019e+06,2.848019,1248.337300
1,10032,2000-10-31,72913210,b'72913210',b'PLEXUS CORP',PLXS,7980.0,11.0,NaN,10437.0,...,36.0,2.0,-0.024480,-0.025155,-0.066833,-0.067922,-0.004949,2.523887e+06,2.523887,1230.292619
2,10032,2001-01-31,72913210,b'72913210',b'PLEXUS CORP',PLXS,7980.0,11.0,NaN,10437.0,...,43.0,2.0,0.039573,0.038787,0.225038,0.224084,0.034637,1.927582e+06,1.927582,1798.386844
3,10078,1990-05-31,86681020,b'86681010',b'SUN MICROSYSTEMS INC',SUNW,8021.0,11.0,NaN,10497.0,...,48.0,2.0,0.088935,0.084648,0.045673,0.043835,0.091989,2.640863e+06,2.640863,1192.735125
4,10078,1990-08-31,86681020,b'86681010',b'SUN MICROSYSTEMS INC',SUNW,8021.0,11.0,NaN,10497.0,...,47.0,2.0,-0.091903,-0.095716,-0.108920,-0.110706,-0.094314,2.662624e+06,2.662624,1080.735500
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
246451,93436,2023-08-31,88160R10,b'88160R10',b'TESLA INC',TSLA,53453.0,11.0,NaN,66252.0,...,55.0,70.0,-0.020605,-0.022263,-0.052542,-0.054050,-0.017716,8.191443e+08,819.144329,645952.785751
246452,93436,2023-09-29,88160R10,b'88160R10',b'TESLA INC',TSLA,53453.0,11.0,NaN,66252.0,...,55.0,70.0,-0.047390,-0.048859,-0.055443,-0.057553,-0.048719,7.954494e+08,795.449384,610422.702658
246453,93436,2023-10-31,88160R10,b'88160R10',b'TESLA INC',TSLA,53453.0,11.0,NaN,66252.0,...,55.0,70.0,-0.028867,-0.029858,-0.060887,-0.062055,-0.021980,6.384545e+08,638.454482,520289.687717
246454,93436,2023-11-30,88160R10,b'88160R10',b'TESLA INC',TSLA,53453.0,11.0,NaN,66252.0,...,54.0,70.0,0.092057,0.090041,0.080251,0.078328,0.089179,7.631954e+08,763.195360,633710.179169


In [51]:
final_cols = ['PERMNO', 'DATE', 'CUSIP', 'COMNAM', 'TICKER', 'start', 'ending',
       'GVKEY', 'TIC', 'COMPANYID', 'LINKDT', 'LINKENDDT']
# Example: convert your date columns to datetime objects
sp500_metadata_df['LINKDT'] = pd.to_datetime(sp500_metadata_df['LINKDT'])
sp500_metadata_df['LINKENDDT'] = pd.to_datetime(sp500_metadata_df['LINKENDDT'])
monthly_metadata_df['DATE'] = pd.to_datetime(monthly_metadata_df['DATE'])

# Fill missing LINKENDDT with the current date (assuming "still valid" means current date)
current_date = pd.to_datetime(datetime.today().date())
sp500_metadata_df['LINKENDDT_filled'] = sp500_metadata_df['LINKENDDT'].fillna(current_date)

# Merge the two dataframes on 'PERMNO'
merged_sp500_monthly_meta_df = pd.merge(monthly_metadata_df, sp500_metadata_df, on='PERMNO', how='left', suffixes=('', '_meta'))

# Filter rows where DATE falls within the [LINKDT, LINKENDDT_filled] interval
mask = (merged_sp500_monthly_meta_df['DATE'] >= merged_sp500_monthly_meta_df['LINKDT']) & (merged_sp500_monthly_meta_df['DATE'] <= merged_sp500_monthly_meta_df['LINKENDDT_filled'])
merged_sp500_monthly_meta_df = merged_sp500_monthly_meta_df.loc[mask]
merged_sp500_monthly_meta_df.reset_index(drop=True, inplace=True)
merged_sp500_monthly_meta_df['COMPANYID'] = merged_sp500_monthly_meta_df['COMPANYID'].astype(int)
merged_sp500_monthly_meta_df = merged_sp500_monthly_meta_df[final_cols]
# Now result_df is your left join where monthly rows are kept only if DATE is within the valid date range.
merged_sp500_monthly_meta_df


,PERMNO,DATE,CUSIP,COMNAM,TICKER,start,ending,GVKEY,TIC,COMPANYID,LINKDT,LINKENDDT
0,10078,1990-05-31,86681020,b'SUN MICROSYSTEMS INC',SUNW,1992-08-20,2010-01-28,012136,JAVA.1,34903,1986-03-04,2010-01-29
1,10078,1990-08-31,86681020,b'SUN MICROSYSTEMS INC',SUNW,1992-08-20,2010-01-28,012136,JAVA.1,34903,1986-03-04,2010-01-29
2,10078,1991-01-31,86681020,b'SUN MICROSYSTEMS INC',SUNW,1992-08-20,2010-01-28,012136,JAVA.1,34903,1986-03-04,2010-01-29
3,10078,1991-02-28,86681020,b'SUN MICROSYSTEMS INC',SUNW,1992-08-20,2010-01-28,012136,JAVA.1,34903,1986-03-04,2010-01-29
4,10078,1991-04-30,86681020,b'SUN MICROSYSTEMS INC',SUNW,1992-08-20,2010-01-28,012136,JAVA.1,34903,1986-03-04,2010-01-29
...,...,...,...,...,...,...,...,...,...,...,...,...
156373,93436,2023-08-31,88160R10,b'TESLA INC',TSLA,2020-12-21,2023-12-29,184996,TSLA,27444752,2010-06-29,NaT
156374,93436,2023-09-29,88160R10,b'TESLA INC',TSLA,2020-12-21,2023-12-29,184996,TSLA,27444752,2010-06-29,NaT
156375,93436,2023-10-31,88160R10,b'TESLA INC',TSLA,2020-12-21,2023-12-29,184996,TSLA,27444752,2010-06-29,NaT
156376,93436,2023-11-30,88160R10,b'TESLA INC',TSLA,2020-12-21,2023-12-29,184996,TSLA,27444752,2010-06-29,NaT


In [52]:
def aggregate_date_ranges(group):
    """
    For each group (with same PERMNO, CUSIP, COMPANYID), sort by DATE and segment 
    the rows into contiguous periods such that the gap between consecutive dates is 
    no more than 6 months (183 days). Returns a DataFrame with aggregated start_date and end_date.
    """
    # Make sure the group is sorted by DATE
    group = group.sort_values('DATE').copy()
    
    # Compute the time difference between consecutive dates
    group['gap'] = group['DATE'] - group['DATE'].shift(1)
    
    # A new segment starts if the gap is more than 183 days or it's the first row (NaT gap)
    group['new_segment'] = (group['gap'] > pd.Timedelta(days=183)) | group['gap'].isna()
    
    # Use cumulative sum to assign a segment ID
    group['segment_id'] = group['new_segment'].cumsum()
    
    # Group by the segment and aggregate the min and max date
    seg_agg = group.groupby('segment_id').agg(
        startdate = ('DATE', 'first'),
        enddate = ('DATE', 'last')
    ).reset_index(drop=True)
    
    # Add the grouping keys back for clarity
    seg_agg['PERMNO'] = group['PERMNO'].iloc[0]
    seg_agg['CUSIP'] = group['CUSIP'].iloc[0]
    seg_agg['COMPANYID'] = group['COMPANYID'].iloc[0]
    seg_agg['TICKER'] = group['TICKER'].iloc[0]
    seg_agg['GVKEY'] = group['GVKEY'].iloc[0]
    
    return seg_agg

# Apply the above function to each group
aggregated_result = merged_sp500_monthly_meta_df.groupby(['PERMNO', 'CUSIP', 'COMPANYID', 'TICKER', 'GVKEY']).apply(aggregate_date_ranges)
# After using groupby.apply, the grouping keys may become part of the index so we reset the index:
aggregated_merged_sp500_monthly_meta_df = aggregated_result.reset_index(drop=True)

aggregated_merged_sp500_monthly_meta_df['startdate'] = aggregated_merged_sp500_monthly_meta_df['startdate'].dt.to_period('M').dt.to_timestamp()
aggregated_merged_sp500_monthly_meta_df['enddate'] = aggregated_merged_sp500_monthly_meta_df['enddate'].dt.to_period('M').dt.to_timestamp('M')


# View the aggregated DataFrame
aggregated_merged_sp500_monthly_meta_df

/tmp/ipykernel_25936/3726760434.py:35: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  aggregated_result = merged_sp500_monthly_meta_df.groupby(['PERMNO', 'CUSIP', 'COMPANYID', 'TICKER', 'GVKEY']).apply(aggregate_date_ranges)


,startdate,enddate,PERMNO,CUSIP,COMPANYID,TICKER,GVKEY
0,2007-08-01,2009-12-31,10078,86681020,34903,JAVA,012136
1,1990-05-01,1992-01-31,10078,86681020,34903,SUNW,012136
2,1992-10-01,1994-03-31,10078,86681020,34903,SUNW,012136
3,1994-10-01,2007-07-31,10078,86681020,34903,SUNW,012136
4,1990-03-01,1990-03-31,10104,68389X10,22247,ORCL,012142
...,...,...,...,...,...,...,...
3219,2016-09-01,2023-12-31,93429,12503M10,32624297,CBOE,184500
3220,2010-07-01,2010-11-30,93436,88160R10,27444752,TSLA,184996
3221,2011-06-01,2011-06-30,93436,88160R10,27444752,TSLA,184996
3222,2012-05-01,2012-05-31,93436,88160R10,27444752,TSLA,184996


In [53]:
surprise_full_df = pd.read_sas("./data/surpsumu1.sas7bdat")
surprise_full_df = surprise_full_df[surprise_full_df['FISCALP'] == b'QTR']
surprise_full_df['TICKER'] = surprise_full_df['TICKER'].apply(lambda x: x.decode('utf-8') if isinstance(x, bytes) else x)
surprise_full_df['OFTIC'] = surprise_full_df['OFTIC'].apply(lambda x: x.decode('utf-8') if isinstance(x, bytes) else x)
surprise_full_df.dropna(subset=['SUESCORE'], inplace=True)

# Function to compute the last day of the given month
def get_last_day(year, month):
    # calendar.monthrange returns a tuple (weekday of first day of the month, number of days in month)
    last_day = calendar.monthrange(year, month)[1]
    return last_day

# Create the 'EPSDATS' column by applying the transformation row-wise
def construct_epsdats(row):
    # Convert PYEAR and PMON from float to int
    year = int(row['PYEAR'])
    month = int(row['PMON'])
    day = get_last_day(year, month)
    # Construct a date object (you can also format it as a string if required)
    return date(year, month, day)

surprise_full_df['EPSDATS'] = surprise_full_df.apply(construct_epsdats, axis=1)

In [54]:
surprise_full_df

,OFTIC,TICKER,MEASURE,ANNDATS,FISCALP,PYEAR,PMON,ACTUAL,SUESCORE,SURPMEAN,SURPSTDEV,USFIRM,EPSDATS
0,TLMR,0000,b'EPS',2014-05-06,b'QTR',2014.0,3.0,0.12,2.82845,0.08000,0.01414,1.0,2014-03-31
1,TLMR,0000,b'EPS',2014-08-06,b'QTR',2014.0,6.0,0.27,11.43137,0.13000,0.01225,1.0,2014-06-30
2,TLMR,0000,b'EPS',2014-11-04,b'QTR',2014.0,9.0,0.26,0.47879,0.23200,0.05848,1.0,2014-09-30
4,TLMR,0000,b'EPS',2015-01-30,b'QTR',2014.0,12.0,0.16,-0.99800,0.17167,0.01169,1.0,2014-12-31
5,TLMR,0000,b'EPS',2015-04-30,b'QTR',2015.0,3.0,0.12,-3.17927,0.17833,0.01835,1.0,2015-03-31
...,...,...,...,...,...,...,...,...,...,...,...,...,...
583413,ZYXI,ZYNX,b'EPS',2019-10-29,b'QTR',2019.0,9.0,0.06,0.57730,0.05667,0.00577,1.0,2019-09-30
583415,ZYXI,ZYNX,b'EPS',2020-02-27,b'QTR',2019.0,12.0,0.09,2.30920,0.07667,0.00577,1.0,2019-12-31
583416,ZYXI,ZYNX,b'EPS',2020-04-28,b'QTR',2020.0,3.0,0.09,2.18549,0.06250,0.01258,1.0,2020-03-31
583417,ZYXI,ZYNX,b'EPS',2020-07-28,b'QTR',2020.0,6.0,0.09,0.29918,0.08640,0.01203,1.0,2020-06-30


In [55]:
surprise_full_df['EPSDATS'] = pd.to_datetime(surprise_full_df['EPSDATS'])
aggregated_merged_sp500_monthly_meta_df['startdate'] = pd.to_datetime(aggregated_merged_sp500_monthly_meta_df['startdate'])
aggregated_merged_sp500_monthly_meta_df['enddate'] = pd.to_datetime(aggregated_merged_sp500_monthly_meta_df['enddate'])

# # Merge based on TICKER (from surprise_full_df) matching meta TICKER
# merge_ticker = pd.merge(
#     surprise_full_df,
#     aggregated_merged_sp500_monthly_meta_df,
#     left_on='TICKER',
#     right_on='TICKER',
#     suffixes=('', '_meta')
# )

# Merge based on OFTIC matching meta TICKER.
merge_oftic = pd.merge(
    surprise_full_df,
    aggregated_merged_sp500_monthly_meta_df,
    left_on='OFTIC',
    right_on='TICKER',
    suffixes=('', '_meta')
)

# Combine the two merge results
# merged_all = pd.concat([merge_ticker, merge_oftic], ignore_index=True)
merged_all  = merge_oftic.copy()

# Apply the date condition: keep rows where ANNDATS is between startdate and enddate
surprise_w_compid_df = merged_all[
    (merged_all['EPSDATS'] >= merged_all['startdate']) &
    (merged_all['EPSDATS'] <= merged_all['enddate'])
]

# Optionally, select only the required columns (including COMPANYID)
surprise_w_compid_df = surprise_w_compid_df[list(surprise_full_df.columns) + ['COMPANYID']].drop_duplicates(subset=['EPSDATS', 'COMPANYID'])
surprise_w_compid_df


,OFTIC,TICKER,MEASURE,ANNDATS,FISCALP,PYEAR,PMON,ACTUAL,SUESCORE,SURPMEAN,SURPSTDEV,USFIRM,EPSDATS,COMPANYID
97,PAYC,003H,b'EPS',2017-02-08,b'QTR',2016.0,12.0,0.18,7.07465,0.12546,0.00771,1.0,2016-12-31,254225545
98,PAYC,003H,b'EPS',2017-05-02,b'QTR',2017.0,3.0,0.47,3.28443,0.41385,0.01710,1.0,2017-03-31,254225545
99,PAYC,003H,b'EPS',2017-08-01,b'QTR',2017.0,6.0,0.26,7.54206,0.20131,0.00778,1.0,2017-06-30,254225545
100,PAYC,003H,b'EPS',2017-10-31,b'QTR',2017.0,9.0,0.29,9.87987,0.18747,0.01038,1.0,2017-09-30,254225545
101,PAYC,003H,b'EPS',2018-05-01,b'QTR',2018.0,3.0,0.95,2.15535,0.89750,0.02436,1.0,2018-03-31,254225545
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
237018,TJX,ZY,b'EPS',2019-11-19,b'QTR',2019.0,10.0,0.68,1.83331,0.65958,0.01114,1.0,2019-10-31,306688
237021,TJX,ZY,b'EPS',2020-02-26,b'QTR',2020.0,1.0,0.81,3.56714,0.76843,0.01165,1.0,2020-01-31,306688
237024,TJX,ZY,b'EPS',2020-05-21,b'QTR',2020.0,4.0,-0.74,-3.83204,-0.17767,0.14674,1.0,2020-04-30,306688
237027,TJX,ZY,b'EPS',2020-08-19,b'QTR',2020.0,7.0,-0.18,-0.39379,-0.09985,0.20353,1.0,2020-07-31,306688


In [56]:
surprise_w_compid_df['sue_standardized'] = (surprise_w_compid_df['SUESCORE'] - surprise_w_compid_df['SUESCORE'].mean()) / surprise_w_compid_df['SUESCORE'].std()

In [58]:
surprise_w_compid_df['SUESCORE'][(surprise_w_compid_df['SUESCORE'] < 0.1) & (surprise_w_compid_df['SUESCORE'] > -0.1)]

110      -0.04100
309       0.02589
1045     -0.05526
1095      0.06656
1110     -0.00815
           ...   
236643    0.06973
236655   -0.07830
236805    0.00000
236841    0.06206
236958    0.02672
Name: SUESCORE, Length: 1958, dtype: float64

In [ ]:
surprise_w_compid_df['sue_standardized'][(surprise_w_compid_df['sue_standardized'] >= 0.1)]

48        0.474326
49        0.949816
220       0.141813
249       0.187176
355       0.196915
            ...   
582575    0.146595
582875    0.212714
582890    0.294119
582976    0.121535
582981    0.107117
Name: sue_standardized, Length: 5583, dtype: float64

In [ ]:
surprise_w_compid_df['SUESCORE'][(surprise_w_compid_df['SUESCORE'] >= 1)]

0          2.82845
1         11.43137
6          1.78891
7          2.12134
8          1.64993
            ...   
583384     1.04237
583387     5.37712
583408     1.44338
583415     2.30920
583416     2.18549
Name: SUESCORE, Length: 179886, dtype: float64

In [ ]:
surprise_w_compid_df['SUESCORE'][(surprise_w_compid_df['SUESCORE'] <= -1)]

5         -3.17927
11        -3.00000
12        -3.66155
22        -2.89012
23        -1.74556
            ...   
583370    -1.11764
583375    -2.97024
583376    -1.07346
583385    -1.12229
583386   -13.83600
Name: SUESCORE, Length: 92792, dtype: float64

In [29]:
processing_year = 2022
start_of_year = pd.Timestamp(f'{processing_year}-01-01')
end_of_year_plus_90 = pd.Timestamp(f'{processing_year}-12-31') + pd.Timedelta(days=90)

# Convert 'EPSDATS' to datetime.
surprise_w_compid_df['EPSDATS'] = pd.to_datetime(surprise_w_compid_df['EPSDATS'])

# Prefilter the surprise dataset.
surprise_w_compid_df[
    (surprise_w_compid_df['EPSDATS'] >= start_of_year) &
    (surprise_w_compid_df['EPSDATS'] < end_of_year_plus_90)
]

,OFTIC,TICKER,MEASURE,ANNDATS,FISCALP,PYEAR,PMON,ACTUAL,SUESCORE,SURPMEAN,SURPSTDEV,USFIRM,EPSDATS,COMPANYID


In [15]:
def processing_transcript_year_order(processing_year, order):
    try:
        # Read the transcript CSV file
        transcript_file = f"./transcript/{processing_year}_{order}.csv"
        year_transcript_df = pd.read_csv(transcript_file)
        logging.info(f"Successfully loaded transcript file: {transcript_file}")
    except Exception as e:
        logging.error(f"Error loading transcript file: {e}")
        return

    try:
        # Build metadata column list, then append additional columns.
        transcript_meta_cols = list(year_transcript_df.columns[1:])  # excluding the first column
        transcript_meta_cols.extend(['PERMNO', 'CUSIP', 'TICKER', 'GVKEY'])
        logging.info("Metadata columns list created.")
    except Exception as e:
        logging.error(f"Error constructing transcript metadata columns: {e}")
        return

    try:
        # Define the columns that uniquely identify a conference event.
        conference_cols = ['companyid', 'mostimportantdateutc', 'mostimportanttimeutc']

        # For each conference, determine the transcriptid to keep (keep the minimum transcriptid).
        chosen_transcripts = (
            year_transcript_df.groupby(conference_cols)['transcriptid']
            .min()
            .reset_index()
            .rename(columns={'transcriptid': 'chosen_transcriptid'})
        )
        logging.info("Chosen transcripts computed using minimum transcriptid.")
    except Exception as e:
        logging.error(f"Error computing chosen transcripts: {e}")
        return

    try:
        # Merge the chosen transcriptid back onto the original DataFrame.
        merged_df = pd.merge(year_transcript_df, chosen_transcripts, on=conference_cols, how='left')

        # Filter the DataFrame to keep only rows where transcriptid matches the chosen_transcriptid.
        year_transcript_df_filtered = merged_df[merged_df['transcriptid'] == merged_df['chosen_transcriptid']].copy()
        
        # Cast columns to int and drop unwanted columns
        year_transcript_df_filtered['companyid'] = year_transcript_df_filtered['companyid'].astype(int)
        year_transcript_df_filtered['transcriptid'] = year_transcript_df_filtered['transcriptid'].astype(int)
        year_transcript_df_filtered['keydevid'] = year_transcript_df_filtered['keydevid'].astype(int)
        year_transcript_df_filtered.drop(columns=['Unnamed: 0'], inplace=True, errors='ignore')
        year_transcript_df_filtered.rename(columns={'companyid': "COMPANYID"}, inplace=True)
        
        # Convert dates to datetime.
        year_transcript_df_filtered['mostimportantdateutc'] = pd.to_datetime(year_transcript_df_filtered['mostimportantdateutc'])
        logging.info("Transcript DataFrame merged and filtered successfully.")
    except Exception as e:
        logging.error(f"Error processing transcript DataFrame: {e}")
        return

    try:
        # These two dataframes should be defined previously or loaded from file.
        # For example, aggregated_merged_sp500_monthly_meta_df might be loaded externally.
        aggregated_merged_sp500_monthly_meta_df['startdate'] = pd.to_datetime(aggregated_merged_sp500_monthly_meta_df['startdate'])
        aggregated_merged_sp500_monthly_meta_df['enddate'] = pd.to_datetime(aggregated_merged_sp500_monthly_meta_df['enddate'])
        logging.info("Loaded SP500 metadata and converted dates.")
    except Exception as e:
        logging.error(f"Error loading or processing SP500 metadata: {e}")
        return

    try:
        # Merge transcripts with metadata based on company id.
        merged = pd.merge(year_transcript_df_filtered, aggregated_merged_sp500_monthly_meta_df, on='COMPANYID', how='inner')
        
        # Define current date for rows where LINKENDDT is NaN (assuming column 'enddate' here).
        current_date = pd.Timestamp.today()
        merged['effective_linkend'] = merged['enddate'].fillna(current_date)
        
        # Filter rows where the transcript date is between startdate and effective_linkend.
        mask = (merged['mostimportantdateutc'] >= merged['startdate']) & (merged['mostimportantdateutc'] <= merged['effective_linkend'])
        merged_filtered = merged.loc[mask].copy()
        
        # Remove duplicates from the transcript side
        transcript_cols = year_transcript_df_filtered.columns.tolist()
        result = merged_filtered.drop_duplicates(subset=transcript_cols, keep='first')
        result['companyid'] = result['COMPANYID']
        result.reset_index(drop=True, inplace=True)
        result = result[transcript_meta_cols]
        logging.info("Merged and filtered transcripts with SP500 metadata.")
    except Exception as e:
        logging.error(f"Error merging and filtering with SP500 metadata: {e}")
        return

    try:
        # Define the date range for the surprise data filter.
        start_of_year = pd.Timestamp(f'{processing_year}-01-01')
        end_of_year_plus_90 = pd.Timestamp(f'{processing_year}-12-31') + pd.Timedelta(days=90)
        
        # Convert 'EPSDATS' to datetime.
        surprise_w_compid_df['EPSDATS'] = pd.to_datetime(surprise_w_compid_df['EPSDATS'])
        
        # Prefilter the surprise dataset.
        surprise_filtered = surprise_w_compid_df[
            (surprise_w_compid_df['EPSDATS'] >= start_of_year) &
            (surprise_w_compid_df['EPSDATS'] < end_of_year_plus_90)
        ].copy()
        logging.info("Filtered surprise dataset based on processing year.")
    except Exception as e:
        logging.error(f"Error processing surprise dataset: {e}")
        return

    try:
        # Ensure that mostimportantdateutc is in datetime format.
        result['mostimportantdateutc'] = pd.to_datetime(result['mostimportantdateutc'])
        
        # Compute matching boundaries.
        result['lower_bound'] = result['mostimportantdateutc'] + pd.DateOffset(months=1)
        result['upper_bound'] = result['mostimportantdateutc'] + pd.Timedelta(days=90)
        
        # Merge the result with the surprise_filtered DataFrame on company id.
        merged = result.merge(surprise_filtered, left_on='companyid', right_on='COMPANYID', how='left')
        
        # Filter merged DataFrame by the matching date condition.
        mask = (merged['EPSDATS'] >= merged['lower_bound']) & (merged['EPSDATS'] < merged['upper_bound'])
        merged_filtered = merged[mask].copy()
        
        # Select the desired result columns.
        # Note: Added a missing comma between 'word_count' and 'ANNDATS'
        res_cols = ['companyid', 'keydevid', 'transcriptid', 'componentorder', 
                    'transcriptcomponenttypeid', 'mostimportantdateutc', 'word_count',
                    'ANNDATS', 'OFTIC', 'EPSDATS', 'ACTUAL', 'SUESCORE', 'SURPMEAN', 
                    'SURPSTDEV', 'componenttext']
        output_df = merged_filtered[res_cols].drop_duplicates(subset=['transcriptid', 'componentorder'])
        logging.info("Successfully merged and filtered all data for transcripts.")
    except Exception as e:
        logging.error(f"Error during final merging and filtering process: {e}")
        return

    try:
        # Write metadata CSV (all columns except 'componenttext')
        metadata_df = output_df.drop(columns=['componenttext'])
        csv_filename = f"./data/train_test_data/transcript_metadata_{processing_year}_{order}.csv"
        metadata_df.to_csv(csv_filename, index=False)
        logging.info(f"Metadata CSV exported as {csv_filename}")
    except Exception as e:
        logging.error(f"Error exporting metadata CSV: {e}")
        return

    try:
        # Write JSONL file for component texts
        jsonl_filename = f"./data/train_test_data/transcript_componenttext_{processing_year}_{order}.jsonl"
        with open(jsonl_filename, 'w', encoding='utf-8') as jsonl_file:
            for _, row in output_df.iterrows():
                # Create key from specified columns using underscore as delimiter.
                key = f"{row['companyid']}_{row['keydevid']}_{row['transcriptid']}_{row['componentorder']}_{row['transcriptcomponenttypeid']}"
                entry = {key: row['componenttext']}
                jsonl_file.write(json.dumps(entry) + "\n")
        logging.info(f"Component text JSONL exported as {jsonl_filename}")
    except Exception as e:
        logging.error(f"Error exporting component text JSONL: {e}")
        return

    logging.info("Processing complete.")

# Example usage:


In [47]:
processing_transcript_year_order(2020, "2")

2025-04-09 12:51:59,840 - INFO - Successfully loaded transcript file: ./transcript/2020_2.csv
2025-04-09 12:51:59,842 - INFO - Metadata columns list created.
2025-04-09 12:52:00,055 - INFO - Chosen transcripts computed using minimum transcriptid.
2025-04-09 12:52:01,288 - INFO - Transcript DataFrame merged and filtered successfully.
2025-04-09 12:52:01,293 - INFO - Loaded SP500 metadata and converted dates.
/tmp/ipykernel_25936/33476018.py:82: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  result['companyid'] = result['COMPANYID']
2025-04-09 12:52:01,839 - INFO - Merged and filtered transcripts with SP500 metadata.
2025-04-09 12:52:01,848 - INFO - Filtered surprise dataset based on processing year.
2025-04-09 12:52:02,011 - INFO - Successfully mer

In [173]:
year_transcript_df = pd.read_csv("./transcript/2010_2.csv")
transcript_meta_cols = year_transcript_df.columns[1:]
transcript_meta_cols = list(transcript_meta_cols) + ['PERMNO', 'CUSIP', 'TICKER', 'GVKEY']

/tmp/ipykernel_1122/4089388992.py:1: DtypeWarning: Columns (25) have mixed types. Specify dtype option on import or set low_memory=False.
  year_transcript_df = pd.read_csv("./transcript/2010_2.csv")


In [174]:
# Define the columns that uniquely identify a conference event.
conference_cols = ['companyid', 'mostimportantdateutc', 'mostimportanttimeutc']

# Step 1: For each conference, determine the transcriptid to keep.
# For example, we keep the minimum transcriptid.
chosen_transcripts = (
    year_transcript_df.groupby(conference_cols)['transcriptid']
      .min()
      .reset_index()
      .rename(columns={'transcriptid': 'chosen_transcriptid'})
)

# Step 2: Merge the chosen transcriptid back onto the original DataFrame.
merged_df = pd.merge(year_transcript_df, chosen_transcripts, on=conference_cols, how='left')

# Step 3: Filter the DataFrame to keep only rows where transcriptid matches the chosen_transcriptid.
year_transcript_df_filtered = merged_df[merged_df['transcriptid'] == merged_df['chosen_transcriptid']].copy()
year_transcript_df_filtered['companyid'] = year_transcript_df_filtered['companyid'].astype(int)
year_transcript_df_filtered['transcriptid'] = year_transcript_df_filtered['transcriptid'].astype(int)
year_transcript_df_filtered['keydevid'] = year_transcript_df_filtered['keydevid'].astype(int)
year_transcript_df_filtered.drop(columns=['Unnamed: 0'], inplace=True)
year_transcript_df_filtered.rename(columns={'companyid':"COMPANYID"}, inplace=True)
year_transcript_df_filtered


,COMPANYID,keydevid,transcriptid,headline,mostimportantdateutc,mostimportanttimeutc,keydeveventtypeid,keydeveventtypename,companyname,transcriptcollectiontypeid,...,transcriptpersonid,transcriptpersonname,proid,companyofperson,speakertypeid,speakertypename,componenttextpreview,word_count,componenttext,chosen_transcriptid
0,250037,107148197,66586,"American Greetings Corp., Q1 2011 Earnings Cal...",2010-06-30,13:00:00,48.0,Earnings Calls,American Greetings Corporation,6,...,1.0,Operator,NaN,NaN,1,Operator,Please standby. We’re about to begin. Good day...,47.0,Please standby. We’re about to begin. Good day...,66586.0
1,250037,107148197,66586,"American Greetings Corp., Q1 2011 Earnings Cal...",2010-06-30,13:00:00,48.0,Earnings Calls,American Greetings Corporation,6,...,207.0,Greg Steinberg,NaN,NaN,2,Executives,"Thank you, Alicia. Good morning, everyone and ...",197.0,"Thank you, Alicia. Good morning, everyone and ...",66586.0
2,250037,107148197,66586,"American Greetings Corp., Q1 2011 Earnings Cal...",2010-06-30,13:00:00,48.0,Earnings Calls,American Greetings Corporation,6,...,654.0,Zev Weiss,NaN,NaN,2,Executives,"Thank you, Greg, and good morning, everyone. T...",1210.0,"Thank you, Greg, and good morning, everyone. T...",66586.0
3,250037,107148197,66586,"American Greetings Corp., Q1 2011 Earnings Cal...",2010-06-30,13:00:00,48.0,Earnings Calls,American Greetings Corporation,6,...,579.0,Steve Smith,37451156.0,NaN,2,Executives,"Thanks, Zev. I have three components to my pre...",1135.0,"Thanks, Zev. I have three components to my pre...",66586.0
4,250037,107148197,66586,"American Greetings Corp., Q1 2011 Earnings Cal...",2010-06-30,13:00:00,48.0,Earnings Calls,American Greetings Corporation,6,...,1.0,Operator,NaN,NaN,1,Operator,"Thank you, sir. (Operator Instructions) We’ll ...",12.0,"Thank you, sir. (Operator Instructions) We’ll ...",66586.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1255799,33593027,109387554,2477213,"PROS Holdings, Inc., Q2 2010 Earnings Call, Au...",2010-08-05,20:30:00,48.0,Earnings Calls,"PROS Holdings, Inc.",7,...,141801.0,Nabil Elsheshai,NaN,NaN,3,Analysts,"Great, and then last question; on the acquisit...",73.0,"Great, and then last question; on the acquisit...",2477213.0
1255800,33593027,109387554,2477213,"PROS Holdings, Inc., Q2 2010 Earnings Call, Au...",2010-08-05,20:30:00,48.0,Earnings Calls,"PROS Holdings, Inc.",7,...,521622.0,Albert Winemiller,NaN,NaN,2,Executives,"Yes, that's a great point. Actually, we have b...",182.0,"Yes, that's a great point. Actually, we have b...",2477213.0
1255801,33593027,109387554,2477213,"PROS Holdings, Inc., Q2 2010 Earnings Call, Au...",2010-08-05,20:30:00,48.0,Earnings Calls,"PROS Holdings, Inc.",7,...,1.0,Operator,NaN,NaN,1,Operator,There are no further questions at this time. T...,32.0,There are no further questions at this time. T...,2477213.0
1255802,33593027,109387554,2477213,"PROS Holdings, Inc., Q2 2010 Earnings Call, Au...",2010-08-05,20:30:00,48.0,Earnings Calls,"PROS Holdings, Inc.",7,...,521622.0,Albert Winemiller,NaN,NaN,2,Executives,"Thank you, operator. In closing, we are please...",211.0,"Thank you, operator. In closing, we are please...",2477213.0


In [175]:
# Make sure dates are in datetime format.
year_transcript_df_filtered['mostimportantdateutc'] = pd.to_datetime(year_transcript_df_filtered['mostimportantdateutc'])
aggregated_merged_sp500_monthly_meta_df['startdate'] = pd.to_datetime(aggregated_merged_sp500_monthly_meta_df['startdate'])
aggregated_merged_sp500_monthly_meta_df['enddate'] = pd.to_datetime(aggregated_merged_sp500_monthly_meta_df['enddate'])

# Merge transcripts with metadata based on company id.
# Notice that transcript df has 'companyid' and metadata has 'COMPANYID'.
merged = pd.merge(year_transcript_df_filtered, aggregated_merged_sp500_monthly_meta_df, on='COMPANYID', how='inner')

# Define current date for rows where LINKENDDT is NaN
current_date = pd.Timestamp.today()

# Create a new column for effective link end date where NaN becomes current_date.
merged['effective_linkend'] = merged['enddate'].fillna(current_date)

# Now filter rows where the transcript date is between LINKDT and effective_linkend.
mask = (merged['mostimportantdateutc'] >= merged['startdate']) & (merged['mostimportantdateutc'] <= merged['effective_linkend'])
merged_filtered = merged.loc[mask].copy()

# Because the same transcript might match multiple metadata rows (if a company appears more than once),
# we want to drop duplicate transcript rows.
# We can use the original columns of the transcript DataFrame to do this.
# Here, we drop duplicates based on the index or on the unique identifier columns available in df_filtered.
transcript_cols = year_transcript_df_filtered.columns.tolist()  # list of columns from the transcript DataFrame
result = merged_filtered.drop_duplicates(subset=transcript_cols, keep='first')
result['companyid'] = result['COMPANYID']
result.reset_index(drop=True, inplace=True)
# The DataFrame "result" now contains only transcript rows where:
# 1. The companyid exists in sp500_metadata_df.
# 2. The mostimportantdateutc is between LINKDT and LINKENDDT (with NaN treated as current).
result = result[transcript_meta_cols]

/tmp/ipykernel_1122/2415153602.py:26: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  result['companyid'] = result['COMPANYID']


In [ ]:
# result[['SUESCORE', 'SURPMEAN', 'SURPSTDEV', 'ANNDATS']] = result.apply(find_matching_surprise, axis=1)
# result

,companyid,keydevid,transcriptid,headline,mostimportantdateutc,mostimportanttimeutc,keydeveventtypeid,keydeveventtypename,companyname,transcriptcollectiontypeid,...,componenttextpreview,word_count,PERMNO,CUSIP,TICKER,GVKEY,SUESCORE,SURPMEAN,SURPSTDEV,ANNDATS
0,528414,108467936,66588,"Monsanto Co., Q3 2010 Earnings Call, Jun 30, 2010",2010-06-30,13:30:00,48.0,Earnings Calls,Monsanto Company,6,...,Welcome to the third quarter 2010 Monsanto Com...,29.0,88668,61166W10,MON,140760,NaN,NaN,NaN,NaT
1,528414,108467936,66588,"Monsanto Co., Q3 2010 Earnings Call, Jun 30, 2010",2010-06-30,13:30:00,48.0,Earnings Calls,Monsanto Company,6,...,Good morning to everybody. I'm on the line tod...,796.0,88668,61166W10,MON,140760,NaN,NaN,NaN,NaT
2,528414,108467936,66588,"Monsanto Co., Q3 2010 Earnings Call, Jun 30, 2010",2010-06-30,13:30:00,48.0,Earnings Calls,Monsanto Company,6,...,"A little more than four weeks ago, we provided...",2258.0,88668,61166W10,MON,140760,NaN,NaN,NaN,NaT
3,528414,108467936,66588,"Monsanto Co., Q3 2010 Earnings Call, Jun 30, 2010",2010-06-30,13:30:00,48.0,Earnings Calls,Monsanto Company,6,...,"Thanks, Carl, and good morning to everybody on...",2171.0,88668,61166W10,MON,140760,NaN,NaN,NaN,NaT
4,528414,108467936,66588,"Monsanto Co., Q3 2010 Earnings Call, Jun 30, 2010",2010-06-30,13:30:00,48.0,Earnings Calls,Monsanto Company,6,...,We'd now like to open the call to your questio...,67.0,88668,61166W10,MON,140760,NaN,NaN,NaN,NaT
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
88350,30956,109085750,1955611,"LSI Corporation, Q2 2010 Earnings Call, Jul-28...",2010-07-28,21:00:00,48.0,Earnings Calls,LSI Corporation,8,...,"Great, thanks for taking my question. Abhi, at...",169.0,48267,50216110,LSI,006529,NaN,NaN,NaN,NaT
88351,30956,109085750,1955611,"LSI Corporation, Q2 2010 Earnings Call, Jul-28...",2010-07-28,21:00:00,48.0,Earnings Calls,LSI Corporation,8,...,My confidence of the company remains equally a...,307.0,48267,50216110,LSI,006529,NaN,NaN,NaN,NaT
88352,30956,109085750,1955611,"LSI Corporation, Q2 2010 Earnings Call, Jul-28...",2010-07-28,21:00:00,48.0,Earnings Calls,LSI Corporation,8,...,"Great, no other question, thanks.",5.0,48267,50216110,LSI,006529,NaN,NaN,NaN,NaT
88353,30956,109085750,1955611,"LSI Corporation, Q2 2010 Earnings Call, Jul-28...",2010-07-28,21:00:00,48.0,Earnings Calls,LSI Corporation,8,...,"Okay, thank you Christian. I believe there are...",43.0,48267,50216110,LSI,006529,NaN,NaN,NaN,NaT


In [176]:
processing_year = 2010

# Define the date range for the surprise_full_df prefilter.
start_of_year = pd.Timestamp(f'{processing_year}-01-01')
end_of_year_plus_90 = pd.Timestamp(f'{processing_year}-12-31') + pd.Timedelta(days=90)

# Convert ANNDATS to datetime if not already converted.
surprise_w_compid_df['EPSDATS'] = pd.to_datetime(surprise_w_compid_df['EPSDATS'])

# Prefilter the surprise dataset to include only rows within the processing year plus 90 days.
surprise_filtered = surprise_w_compid_df[
    (surprise_w_compid_df['EPSDATS'] >= start_of_year) &
    (surprise_w_compid_df['EPSDATS'] < end_of_year_plus_90)
].copy()
surprise_filtered

# # Melt the surprise table so that the two ticker columns are unified.
# cols_to_keep = [col for col in surprise_filtered.columns if col not in ['TICKER', 'OFTIC']]
# surprise_melted = surprise_filtered.melt(
#     id_vars=cols_to_keep,
#     value_vars=['TICKER', 'OFTIC'],
#     var_name='ticker_field',
#     value_name='join_ticker'
# )

# Convert the date in result, if necessary.
result['mostimportantdateutc'] = pd.to_datetime(result['mostimportantdateutc'])

# Compute the matching boundaries for each row in result.
result['lower_bound'] = result['mostimportantdateutc'] + pd.DateOffset(months=1)
result['upper_bound'] = result['mostimportantdateutc'] + pd.Timedelta(days=90)

In [178]:
print(result.merge(surprise_filtered, left_on='companyid', right_on='COMPANYID', how='left').transcriptid.nunique())
# Merge result with the melted surprise table on ticker equality.
merged = result.merge(surprise_filtered, left_on='companyid', right_on='COMPANYID', how='left')

# Filter the merged DataFrame by the matching date condition.
# mask = (merged['ANNDATS'] >= merged['lower_bound']) & (merged['ANNDATS'] < merged['upper_bound'])
# merged_filtered = merged[mask].copy()
merged_filtered = merged.copy()
merged_filtered[['companyid', 'keydevid', 'transcriptid', 'mostimportantdateutc', 'lower_bound', 'upper_bound', 'ANNDATS', 'OFTIC', 'EPSDATS']]

1022


,companyid,keydevid,transcriptid,mostimportantdateutc,lower_bound,upper_bound,ANNDATS,OFTIC,EPSDATS
0,528414,108467936,66588,2010-06-30,2010-07-30,2010-09-28,2010-04-07,MON,2010-02-28
1,528414,108467936,66588,2010-06-30,2010-07-30,2010-09-28,2010-06-30,MON,2010-05-31
2,528414,108467936,66588,2010-06-30,2010-07-30,2010-09-28,2010-10-06,MON,2010-08-31
3,528414,108467936,66588,2010-06-30,2010-07-30,2010-09-28,2011-01-06,MON,2010-11-30
4,528414,108467936,66588,2010-06-30,2010-07-30,2010-09-28,2011-04-06,MON,2011-02-28
...,...,...,...,...,...,...,...,...,...
355709,30956,109085750,1955611,2010-07-28,2010-08-28,2010-10-26,2010-07-28,LSI,2010-06-30
355710,30956,109085750,1955611,2010-07-28,2010-08-28,2010-10-26,2010-04-28,LSI,2010-03-31
355711,30956,109085750,1955611,2010-07-28,2010-08-28,2010-10-26,2010-07-28,LSI,2010-06-30
355712,30956,109085750,1955611,2010-07-28,2010-08-28,2010-10-26,2010-04-28,LSI,2010-03-31


In [179]:
# Filter the merged DataFrame by the matching date condition.
mask = (merged['EPSDATS'] >= merged['lower_bound']) & (merged['EPSDATS'] < merged['upper_bound'])
merged_filtered = merged[mask].copy()
merged_filtered[['companyid', 'keydevid', 'transcriptid', 'componentorder', 'transcriptcomponenttypeid', 'mostimportantdateutc', 'lower_bound', 'upper_bound', 'ANNDATS', 'OFTIC', 'EPSDATS', 'ACTUAL','SUESCORE','SURPMEAN','SURPSTDEV', ]].drop_duplicates(subset=['transcriptid', 'componentorder'])

,companyid,keydevid,transcriptid,componentorder,transcriptcomponenttypeid,mostimportantdateutc,lower_bound,upper_bound,ANNDATS,OFTIC,EPSDATS,ACTUAL,SUESCORE,SURPMEAN,SURPSTDEV
2,528414,108467936,66588,1,1,2010-06-30,2010-07-30,2010-09-28,2010-10-06,MON,2010-08-31,-0.09,-1.59481,-0.0600,0.01881
7,528414,108467936,66588,2,2,2010-06-30,2010-07-30,2010-09-28,2010-10-06,MON,2010-08-31,-0.09,-1.59481,-0.0600,0.01881
12,528414,108467936,66588,3,2,2010-06-30,2010-07-30,2010-09-28,2010-10-06,MON,2010-08-31,-0.09,-1.59481,-0.0600,0.01881
17,528414,108467936,66588,4,2,2010-06-30,2010-07-30,2010-09-28,2010-10-06,MON,2010-08-31,-0.09,-1.59481,-0.0600,0.01881
22,528414,108467936,66588,5,2,2010-06-30,2010-07-30,2010-09-28,2010-10-06,MON,2010-08-31,-0.09,-1.59481,-0.0600,0.01881
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
352168,306688,116908688,403610,41,4,2010-12-10,2011-01-10,2011-03-10,2011-02-23,TJX,2011-01-31,1.05,7.78051,1.0215,0.00366
352173,306688,116908688,403610,42,3,2010-12-10,2011-01-10,2011-03-10,2011-02-23,TJX,2011-01-31,1.05,7.78051,1.0215,0.00366
352178,306688,116908688,403610,43,4,2010-12-10,2011-01-10,2011-03-10,2011-02-23,TJX,2011-01-31,1.05,7.78051,1.0215,0.00366
352183,306688,116908688,403610,44,4,2010-12-10,2011-01-10,2011-03-10,2011-02-23,TJX,2011-01-31,1.05,7.78051,1.0215,0.00366


In [180]:
merged_filtered[['companyid', 'keydevid', 'transcriptid', 'componentorder', 'transcriptcomponenttypeid', 'mostimportantdateutc', 'lower_bound', 'upper_bound', 'ANNDATS', 'OFTIC', 'EPSDATS', 'ACTUAL','SUESCORE','SURPMEAN','SURPSTDEV', 'word_count', 'componenttext']]

,companyid,keydevid,transcriptid,componentorder,transcriptcomponenttypeid,mostimportantdateutc,lower_bound,upper_bound,ANNDATS,OFTIC,EPSDATS,ACTUAL,SUESCORE,SURPMEAN,SURPSTDEV,word_count,componenttext
2,528414,108467936,66588,1,1,2010-06-30,2010-07-30,2010-09-28,2010-10-06,MON,2010-08-31,-0.09,-1.59481,-0.0600,0.01881,29.0,Welcome to the third quarter 2010 Monsanto Com...
7,528414,108467936,66588,2,2,2010-06-30,2010-07-30,2010-09-28,2010-10-06,MON,2010-08-31,-0.09,-1.59481,-0.0600,0.01881,796.0,Good morning to everybody. I'm on the line tod...
12,528414,108467936,66588,3,2,2010-06-30,2010-07-30,2010-09-28,2010-10-06,MON,2010-08-31,-0.09,-1.59481,-0.0600,0.01881,2258.0,"A little more than four weeks ago, we provided..."
17,528414,108467936,66588,4,2,2010-06-30,2010-07-30,2010-09-28,2010-10-06,MON,2010-08-31,-0.09,-1.59481,-0.0600,0.01881,2171.0,"Thanks, Carl, and good morning to everybody on..."
22,528414,108467936,66588,5,2,2010-06-30,2010-07-30,2010-09-28,2010-10-06,MON,2010-08-31,-0.09,-1.59481,-0.0600,0.01881,67.0,We'd now like to open the call to your questio...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
352168,306688,116908688,403610,41,4,2010-12-10,2011-01-10,2011-03-10,2011-02-23,TJX,2011-01-31,1.05,7.78051,1.0215,0.00366,11.0,"Yes, slightly higher but below comparable Marm..."
352173,306688,116908688,403610,42,3,2010-12-10,2011-01-10,2011-03-10,2011-02-23,TJX,2011-01-31,1.05,7.78051,1.0215,0.00366,21.0,"Okay, do you plan to convert TJX Europe conver..."
352178,306688,116908688,403610,43,4,2010-12-10,2011-01-10,2011-03-10,2011-02-23,TJX,2011-01-31,1.05,7.78051,1.0215,0.00366,5.0,They will be new stores.
352183,306688,116908688,403610,44,4,2010-12-10,2011-01-10,2011-03-10,2011-02-23,TJX,2011-01-31,1.05,7.78051,1.0215,0.00366,24.0,"Well, thank you, everyone and we appreciate yo..."


In [ ]:
processing_year = 2010

# Define the date range for the surprise_full_df prefilter.
start_of_year = pd.Timestamp(f'{processing_year}-01-01')
end_of_year_plus_90 = pd.Timestamp(f'{processing_year}-12-31') + pd.Timedelta(days=90)

# Convert ANNDATS to datetime if not already converted.
surprise_full_df['ANNDATS'] = pd.to_datetime(surprise_full_df['ANNDATS'])

# Prefilter the surprise dataset to include only rows within the processing year plus 90 days.
surprise_filtered = surprise_full_df[
    (surprise_full_df['ANNDATS'] >= start_of_year) &
    (surprise_full_df['ANNDATS'] < end_of_year_plus_90)
].copy()

# ------------------------
# Continue with the vectorized matching process.

# Convert the date in result, if necessary.
result['mostimportantdateutc'] = pd.to_datetime(result['mostimportantdateutc'])

# Compute the matching boundaries for each row in result.
result['lower_bound'] = result['mostimportantdateutc'] + pd.DateOffset(months=1)
result['upper_bound'] = result['mostimportantdateutc'] + pd.Timedelta(days=90)

# Melt the surprise table so that the two ticker columns are unified.
cols_to_keep = [col for col in surprise_filtered.columns if col not in ['TICKER', 'OFTIC']]
surprise_melted = surprise_filtered.melt(
    id_vars=cols_to_keep,
    value_vars=['TICKER', 'OFTIC'],
    var_name='ticker_field',
    value_name='join_ticker'
)

# Merge result with the melted surprise table on ticker equality.
merged = result.merge(surprise_melted, left_on='TICKER', right_on='join_ticker', how='left')

# Filter the merged DataFrame by the matching date condition.
mask = (merged['ANNDATS'] >= merged['lower_bound']) & (merged['ANNDATS'] < merged['upper_bound'])
merged_filtered = merged[mask].copy()

# Save the original index to group back the candidates.
merged_filtered['orig_index'] = merged_filtered.index

In [52]:
merged_filtered.transcriptid

2           15674
7           15674
12          15674
17          15674
22          15674
           ...   
948515    1364542
948521    1364542
948527    1364542
948533    1364542
948539    1364542
Name: transcriptid, Length: 56322, dtype: int64

In [23]:
processing_year = 2010

# Define the date range for the surprise_full_df prefilter.
start_of_year = pd.Timestamp(f'{processing_year}-01-01')
end_of_year_plus_90 = pd.Timestamp(f'{processing_year}-12-31') + pd.Timedelta(days=90)

# Convert ANNDATS to datetime if not already converted.
surprise_full_df['ANNDATS'] = pd.to_datetime(surprise_full_df['ANNDATS'])

# Prefilter the surprise dataset to include only rows within the processing year plus 90 days.
surprise_filtered = surprise_full_df[
    (surprise_full_df['ANNDATS'] >= start_of_year) &
    (surprise_full_df['ANNDATS'] < end_of_year_plus_90)
].copy()

# ------------------------
# Continue with the vectorized matching process.

# Convert the date in result, if necessary.
result['mostimportantdateutc'] = pd.to_datetime(result['mostimportantdateutc'])

# Compute the matching boundaries for each row in result.
result['lower_bound'] = result['mostimportantdateutc'] + pd.DateOffset(months=1)
result['upper_bound'] = result['mostimportantdateutc'] + pd.Timedelta(days=90)

# Melt the surprise table so that the two ticker columns are unified.
cols_to_keep = [col for col in surprise_filtered.columns if col not in ['TICKER', 'OFTIC']]
surprise_melted = surprise_filtered.melt(
    id_vars=cols_to_keep,
    value_vars=['TICKER', 'OFTIC'],
    var_name='ticker_field',
    value_name='join_ticker'
)

# Merge result with the melted surprise table on ticker equality.
merged = result.merge(surprise_melted, left_on='TICKER', right_on='join_ticker', how='left')

# Filter the merged DataFrame by the matching date condition.
mask = (merged['ANNDATS'] >= merged['lower_bound']) & (merged['ANNDATS'] < merged['upper_bound'])
merged_filtered = merged[mask].copy()

# Save the original index to group back the candidates.
merged_filtered['orig_index'] = merged_filtered.index

# Group by original row index and select the candidate with the earliest ANNDATS.
best_matches = (
    merged_filtered.sort_values('ANNDATS')  # sort so that the earliest ANNDATS is at the top
    .groupby('orig_index', as_index=False)
    .first()  # select the first candidate for each group
)

# Merge the best matches back to the original result DataFrame.
result_with_surprise = result.merge(
    best_matches[['orig_index', 'SUESCORE', 'SURPMEAN', 'SURPSTDEV', 'ANNDATS']],
    left_index=True, right_on='orig_index', how='left'
)

# (Optional) Drop the extra columns used for the join.
result_with_surprise = result_with_surprise.drop(columns=['lower_bound', 'upper_bound'])

# Check the final DataFrame
result_with_surprise

,companyid,keydevid,transcriptid,headline,mostimportantdateutc,mostimportanttimeutc,keydeveventtypeid,keydeveventtypename,companyname,transcriptcollectiontypeid,...,word_count,PERMNO,CUSIP,TICKER,GVKEY,orig_index,SUESCORE,SURPMEAN,SURPSTDEV,ANNDATS
NaN,528414,108467936,66588,"Monsanto Co., Q3 2010 Earnings Call, Jun 30, 2010",2010-06-30,13:30:00,48.0,Earnings Calls,Monsanto Company,6,...,29.0,88668,61166W10,MON,140760,0,NaN,NaN,NaN,NaT
NaN,528414,108467936,66588,"Monsanto Co., Q3 2010 Earnings Call, Jun 30, 2010",2010-06-30,13:30:00,48.0,Earnings Calls,Monsanto Company,6,...,796.0,88668,61166W10,MON,140760,1,NaN,NaN,NaN,NaT
NaN,528414,108467936,66588,"Monsanto Co., Q3 2010 Earnings Call, Jun 30, 2010",2010-06-30,13:30:00,48.0,Earnings Calls,Monsanto Company,6,...,2258.0,88668,61166W10,MON,140760,2,NaN,NaN,NaN,NaT
NaN,528414,108467936,66588,"Monsanto Co., Q3 2010 Earnings Call, Jun 30, 2010",2010-06-30,13:30:00,48.0,Earnings Calls,Monsanto Company,6,...,2171.0,88668,61166W10,MON,140760,3,NaN,NaN,NaN,NaT
NaN,528414,108467936,66588,"Monsanto Co., Q3 2010 Earnings Call, Jun 30, 2010",2010-06-30,13:30:00,48.0,Earnings Calls,Monsanto Company,6,...,67.0,88668,61166W10,MON,140760,4,NaN,NaN,NaN,NaT
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
NaN,30956,109085750,1955611,"LSI Corporation, Q2 2010 Earnings Call, Jul-28...",2010-07-28,21:00:00,48.0,Earnings Calls,LSI Corporation,8,...,169.0,48267,50216110,LSI,006529,88350,NaN,NaN,NaN,NaT
NaN,30956,109085750,1955611,"LSI Corporation, Q2 2010 Earnings Call, Jul-28...",2010-07-28,21:00:00,48.0,Earnings Calls,LSI Corporation,8,...,307.0,48267,50216110,LSI,006529,88351,NaN,NaN,NaN,NaT
NaN,30956,109085750,1955611,"LSI Corporation, Q2 2010 Earnings Call, Jul-28...",2010-07-28,21:00:00,48.0,Earnings Calls,LSI Corporation,8,...,5.0,48267,50216110,LSI,006529,88352,NaN,NaN,NaN,NaT
NaN,30956,109085750,1955611,"LSI Corporation, Q2 2010 Earnings Call, Jul-28...",2010-07-28,21:00:00,48.0,Earnings Calls,LSI Corporation,8,...,43.0,48267,50216110,LSI,006529,88353,NaN,NaN,NaN,NaT


In [24]:
result_with_surprise.dropna(subset=['SUESCORE'])['transcriptid'].nunique()

94

In [10]:
all_tickers_df = pd.read_csv("tickers_all_with_companyid.csv")
df1 = pd.read_csv("./transcript_metadata_2010_1.csv")
df2 = pd.read_csv("./transcript_metadata_2010_2.csv")
df = pd.concat([df1, df2])
df['companyid'] = df["companyid"].astype(int)
temp_df = df[df['companyid'].isin(list(all_tickers_df.loc[all_tickers_df['TICKER'].isin(ticker_list), 'COMPANYID'].unique()))]
temp_df.companyid.nunique()

/tmp/ipykernel_575659/1266356600.py:1: DtypeWarning: Columns (14) have mixed types. Specify dtype option on import or set low_memory=False.
  all_tickers_df = pd.read_csv("tickers_all_with_companyid.csv")


614

In [50]:
df[df['companyid'].isin(list(all_tickers_df.loc[all_tickers_df['TICKER'].isin(monthly_metadata_df.TICKER.to_list()), 'COMPANYID'].unique()))]

,companyid,keydevid,transcriptid,componentorder,transcriptcomponentid,transcriptcomponenttypeid,mostimportantdateutc
0,405869,84102073.0,15674.0,1,13334555.0,1,2010-01-27
1,405869,84102073.0,15674.0,2,13334556.0,2,2010-01-27
2,405869,84102073.0,15674.0,3,13334557.0,2,2010-01-27
3,405869,84102073.0,15674.0,4,13334558.0,2,2010-01-27
4,405869,84102073.0,15674.0,5,13334559.0,2,2010-01-27
...,...,...,...,...,...,...,...
541796,314896,113942632.0,2713425.0,77,101814920.0,4,2010-10-26
541797,314896,113942632.0,2713425.0,78,101814921.0,7,2010-10-26
541798,314896,113942632.0,2713425.0,79,101814922.0,4,2010-10-26
541799,314896,113942632.0,2713425.0,80,101814923.0,4,2010-10-26


In [37]:
temp_df.sort_values(by=['companyid', 'mostimportantdateutc']).to_csv("temp.csv", index=False)

In [45]:
temp_df.groupby(['companyid', 'mostimportantdateutc']).size().reset_index().shape[0]

4211

In [38]:
temp_df.groupby('companyid')['transcriptid'].nunique()

companyid
18749        9
19049       20
19691       12
21127       14
21835       19
22247       22
24937       12
27807       10
30771       10
32307       13
94238        9
100231       9
106335      22
107534      11
139677      14
139813      16
162270      10
168569       6
168864      12
177031      17
190694       8
205573       9
250388       2
258823      12
260149      13
266311      10
270961      11
271409      10
273607      12
286798       8
289030      17
292891      13
293286      15
304260      15
305304      15
309845       4
314842      13
317627      15
318091       8
323857       9
332124      12
370857       9
382914       4
391687      16
399960      16
405483      11
406338      13
415798      12
472898      17
658776      16
1342560     11
3632895     10
13580386     6
23812789     3
27444752     3
Name: transcriptid, dtype: int64

In [24]:
all_tickers_df = pd.read_csv("tickers_big_mid_cap_with_companyid.csv")
all_tickers_df.loc[all_tickers_df['TICKER'].isin(monthly_metadata_df.TICKER.to_list()), 'COMPANYID'].unique()

array([    168569,     168864,     250388,      24937,     415798,
           370857,     260149,     658776,     391687,     266311,
        158387416,     406338,     318091,     270961,     271409,
           305304,     106335,     177031,      21127,     139677,
           179862,   34952431,     314842,     289030,      19049,
            21985,     292891,     293286,     139813,     162270,
           100231,     304260,     107534,     309845,     472898,
            21835,      22247,      30771,     258823,      34093,
           286798,     190694,     273607,  212741644,   35962803,
            19691,  265511894,  272485300,     205573,     317627,
           332124,      34205,  369991357,     323857,  224055283,
            94238,  551009282,  144524848,   25116311,  590935798,
        675705521,   43580005,  243722644,  321227313,  141582707,
         83747444,  144746614, 1820759186,      27807,      18749,
           382914,     399960,      28922,    4180463,      32

In [3]:
all_tickers_df = pd.read_csv("tickers_all.csv")
gvkey_df = pd.read_csv("gvkey_companyid.csv")
all_tickers_df['GVKEY'] = all_tickers_df['GVKEY'].astype(str).str.zfill(6)
gvkey_df['GVKEY'] = gvkey_df['GVKEY'].astype(str).str.zfill(6)
merged = pd.merge(all_tickers_df, gvkey_df[['COMPANYID', 'GVKEY', 'ENDDATE']], on='GVKEY', how='left')
filtered = merged[(merged['ENDDATE'].isna()) | (merged['DATADATE'] < merged['ENDDATE'])]
# filtered.to_csv("temp.csv")
filtered

,GVKEY,TICKER,DATADATE,FYEAR,CSHTR_C,CSHTR_F,PRCH_C,PRCL_C,PRCC_C,MKVALT,est_avg_price,calendar_avg,fiscal_avg,COMPANYID,ENDDATE
0,001000,AE.2,1970-12-31,1970.0,9.600000e+04,9.590000e+04,31.000000,6.750,10.000,NaN,15.916667,3.809524e+02,3.805556e+02,5.356281e+07,NaN
1,001000,AE.2,1971-12-31,1971.0,1.123200e+06,1.123200e+06,12.875000,3.625,5.750,NaN,7.416667,4.457143e+03,4.457143e+03,5.356281e+07,NaN
2,001000,AE.2,1972-12-31,1972.0,1.008100e+06,1.008100e+06,8.750000,4.625,5.125,NaN,6.166667,4.000397e+03,4.000397e+03,5.356281e+07,NaN
3,001000,AE.2,1973-12-31,1973.0,4.338000e+05,4.338000e+05,5.500000,1.125,1.750,NaN,2.791667,1.721429e+03,1.721429e+03,5.356281e+07,NaN
4,001000,AE.2,1974-12-31,1974.0,1.856000e+05,1.856000e+05,3.375000,1.875,2.125,NaN,2.458333,7.365079e+02,7.365079e+02,5.356281e+07,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
346552,351038,QNRX,2021-12-31,2021.0,6.220461e+07,6.220461e+07,13.455000,1.700,1.820,NaN,5.658333,2.468437e+05,2.468437e+05,5.930108e+08,NaN
346553,351038,QNRX,2022-12-31,2022.0,2.031618e+08,2.031618e+08,29.375000,1.110,1.420,NaN,10.635000,8.061976e+05,8.061976e+05,5.930108e+08,NaN
346554,351038,QNRX,2023-12-31,2023.0,1.682911e+07,1.682911e+07,33.840001,3.640,4.905,NaN,14.128334,6.678218e+04,6.678218e+04,5.930108e+08,NaN
346555,352262,CLCO,2023-12-31,2023.0,2.585268e+07,2.585268e+07,14.500000,11.000,12.720,683.1022,12.740000,1.025900e+05,1.025900e+05,5.924039e+08,NaN


In [5]:
filtered.drop_duplicates(subset=['GVKEY', 'DATADATE']).to_csv("tickers_with_companyid.csv", index=False)

In [6]:
filtered.drop_duplicates(subset=['GVKEY', 'DATADATE'])['GVKEY'].nunique()

27174

In [5]:
gvkey_df.to_csv("gvkey_companyid.csv", index=False)

In [2]:
met_df = pd.read_sas("./data/ccmfunda.sas7bdat")
met_df 

,GVKEY,DATADATE,FYEAR,ADD1,ADD2,ADD3,ADD4,ADDZIP,BUSDESC,CITY,...,PRCH_C,PRCH_F,PRCL_C,PRCL_F,CONSOL,INDFMT,DATAFMT,POPSRC,CURCD,COSTAT
0,b'001000',1970-12-31,1970.0,NaN,NaN,NaN,NaN,NaN,b'A & E Plastik Pak Inc. is a commodity chemic...,NaN,...,31.000000,10.875000,6.750,7.500,b'C',b'INDL',b'STD',b'D',b'USD',b'I'
1,b'001000',1971-12-31,1971.0,NaN,NaN,NaN,NaN,NaN,b'A & E Plastik Pak Inc. is a commodity chemic...,NaN,...,12.875000,12.875000,3.625,3.625,b'C',b'INDL',b'STD',b'D',b'USD',b'I'
2,b'001000',1972-12-31,1972.0,NaN,NaN,NaN,NaN,NaN,b'A & E Plastik Pak Inc. is a commodity chemic...,NaN,...,8.750000,8.750000,4.625,4.625,b'C',b'INDL',b'STD',b'D',b'USD',b'I'
3,b'001000',1973-12-31,1973.0,NaN,NaN,NaN,NaN,NaN,b'A & E Plastik Pak Inc. is a commodity chemic...,NaN,...,5.500000,5.500000,1.125,1.125,b'C',b'INDL',b'STD',b'D',b'USD',b'I'
4,b'001000',1974-12-31,1974.0,NaN,NaN,NaN,NaN,NaN,b'A & E Plastik Pak Inc. is a commodity chemic...,NaN,...,3.375000,3.375000,1.875,1.875,b'C',b'INDL',b'STD',b'D',b'USD',b'I'
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
330742,b'351038',2021-12-31,2021.0,b'42127 Pleasant Forest Court',NaN,NaN,NaN,b'20148-7349',"b'Quoin Pharmaceuticals, Ltd., a clinical stag...",b'Ashburn',...,13.455000,13.455000,1.700,1.700,b'C',b'INDL',b'STD',b'D',b'USD',b'A'
330743,b'351038',2022-12-31,2022.0,b'42127 Pleasant Forest Court',NaN,NaN,NaN,b'20148-7349',"b'Quoin Pharmaceuticals, Ltd., a clinical stag...",b'Ashburn',...,29.375000,29.375000,1.110,1.110,b'C',b'INDL',b'STD',b'D',b'USD',b'A'
330744,b'351038',2023-12-31,2023.0,b'42127 Pleasant Forest Court',NaN,NaN,NaN,b'20148-7349',"b'Quoin Pharmaceuticals, Ltd., a clinical stag...",b'Ashburn',...,33.840001,33.840001,3.640,3.640,b'C',b'INDL',b'STD',b'D',b'USD',b'A'
330745,b'352262',2023-12-31,2023.0,"b'7 Clarges Street, 5th Floor'",NaN,NaN,NaN,b'W1J 8AE',b'Cool Company Ltd. engages in the acquisition...,b'London',...,14.500000,14.500000,11.000,11.000,b'C',b'INDL',b'STD',b'D',b'USD',b'A'


In [15]:
traded_vol_df = met_df[['GVKEY', 'TIC', 'DATADATE', 'FYEAR', 'CSHTR_C', 'CSHTR_F', 'PRCH_C', 'PRCL_C', 'PRCC_C', 'MKVALT']]
traded_vol_df['est_avg_price'] = (traded_vol_df['PRCH_C'] + traded_vol_df['PRCL_C'] + traded_vol_df['PRCC_C']) / 3
traded_vol_df['calendar_avg'] = traded_vol_df['CSHTR_C'] / 252
traded_vol_df['fiscal_avg'] = traded_vol_df['CSHTR_F'] / 252

/tmp/ipykernel_240969/1208575187.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  traded_vol_df['est_avg_price'] = (traded_vol_df['PRCH_C'] + traded_vol_df['PRCL_C'] + traded_vol_df['PRCC_C']) / 3
/tmp/ipykernel_240969/1208575187.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  traded_vol_df['calendar_avg'] = traded_vol_df['CSHTR_C'] / 252
/tmp/ipykernel_240969/1208575187.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_in

In [16]:
traded_vol_df['TIC'] = traded_vol_df.TIC.apply(lambda x: x.decode('utf-8') if isinstance(x, bytes) else x)
traded_vol_df['GVKEY'] = traded_vol_df.GVKEY.apply(lambda x: x.decode('utf-8') if isinstance(x, bytes) else x)
traded_vol_df = traded_vol_df.rename(columns={"TIC":"TICKER"})
traded_vol_df.to_csv("tickers_all.csv", index=False)

/tmp/ipykernel_240969/3144586771.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  traded_vol_df['TIC'] = traded_vol_df.TIC.apply(lambda x: x.decode('utf-8') if isinstance(x, bytes) else x)
/tmp/ipykernel_240969/3144586771.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  traded_vol_df['GVKEY'] = traded_vol_df.GVKEY.apply(lambda x: x.decode('utf-8') if isinstance(x, bytes) else x)


In [11]:
all_tickers_df = traded_vol_df[(traded_vol_df['fiscal_avg'] * traded_vol_df['est_avg_price'] >= 50000000) & (traded_vol_df['MKVALT'] * 1000000 >= 1000000000)]

In [14]:
all_tickers_df['TIC'] = all_tickers_df.TIC.apply(lambda x: x.decode('utf-8') if isinstance(x, bytes) else x)
all_tickers_df['GVKEY'] = all_tickers_df.GVKEY.apply(lambda x: x.decode('utf-8') if isinstance(x, bytes) else x)
all_tickers_df = all_tickers_df.rename(columns={"TIC":"TICKER"})
ticker_set = set(all_tickers_df['TICKER'].to_list())

AttributeError: 'DataFrame' object has no attribute 'TIC'

In [14]:
all_tickers_df.to_csv("tickers_big_mid_cap.csv", index=False)

In [13]:
all_tickers_df

1994

In [ ]:
# metadata_df = pd.read_sas("./data/msfepssurprise.sas7bdat")

# metadata_df.TICKER = metadata_df.TICKER.apply(lambda x: x.decode('utf-8') if isinstance(x, bytes) else x)

# ticker_set = set(metadata_df['TICKER'].to_list())

In [ ]:
# len(ticker_set)

169

In [ ]:
# len(set(metadata_df.loc[(metadata_df.DATE.dt.date >= date(2004, 1, 1)) &(metadata_df.DATE.dt.date < date(2010, 1, 1)) , 'TICKER'].to_list()))

64

In [13]:
surprise_df = pd.read_sas("./data/surpsumu1.sas7bdat")
surprise_df = surprise_df[surprise_df['FISCALP'] == b'QTR']
surprise_df.TICKER = surprise_df.TICKER.apply(lambda x: x.decode('utf-8') if isinstance(x, bytes) else x)
surprise_df.OFTIC = surprise_df.OFTIC.apply(lambda x: x.decode('utf-8') if isinstance(x, bytes) else x)
surprise_df = surprise_df[(surprise_df['TICKER'].isin(ticker_set)) | (surprise_df['OFTIC'].isin(ticker_set))]
surprise_df = surprise_df[surprise_df['USFIRM'] > 0]

NameError: name 'ticker_set' is not defined

In [14]:
subset_df = surprise_df[(surprise_df.ANNDATS.dt.date >= date(2004, 1, 1)) & (surprise_df.ANNDATS.dt.date < date(2010, 1, 1))]
subset_df.TICKER.nunique()

1238

In [15]:
subset_df

,OFTIC,TICKER,MEASURE,ANNDATS,FISCALP,PYEAR,PMON,ACTUAL,SUESCORE,SURPMEAN,SURPSTDEV,USFIRM
8108,AA,AA,b'EPS',2004-01-08,b'QTR',2003.0,12.0,0.27,-3.62357,0.34353,0.02029,1.0
8109,AA,AA,b'EPS',2004-04-06,b'QTR',2004.0,3.0,0.37,-1.88648,0.41529,0.02401,1.0
8110,AA,AA,b'EPS',2004-07-07,b'QTR',2004.0,6.0,0.46,-0.36878,0.46938,0.02542,1.0
8111,AA,AA,b'EPS',2004-10-07,b'QTR',2004.0,9.0,0.34,0.42817,0.33500,0.01168,1.0
8113,AA,AA,b'EPS',2005-01-10,b'QTR',2004.0,12.0,0.39,-0.49546,0.41053,0.04143,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...
583301,TJX,ZY,b'EPS',2008-11-11,b'QTR',2008.0,10.0,0.57,1.80899,0.54571,0.01343,1.0
583303,TJX,ZY,b'EPS',2009-02-25,b'QTR',2009.0,1.0,0.55,8.53890,0.50500,0.00527,1.0
583304,TJX,ZY,b'EPS',2009-05-19,b'QTR',2009.0,4.0,0.49,0.71649,0.48647,0.00493,1.0
583305,TJX,ZY,b'EPS',2009-08-18,b'QTR',2009.0,7.0,0.61,1.84913,0.59579,0.00769,1.0


In [37]:
df = pd.read_csv('./transcript/2010_1.csv', nrows=1000000)
# df.to_csv("sample_transcript_5000.csv", index=False)

/tmp/ipykernel_242029/1780567082.py:1: DtypeWarning: Columns (25) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv('./transcript/2010_1.csv', nrows=1000000)


In [43]:
df.transcriptid.nunique()

5265

In [41]:
df = pd.read_csv("transcript_metadata_2010_1.csv")
df2 = pd.read_csv("transcript_metadata_2010_2.csv")
combined_df = pd.concat([df, df2])
combined_df.companyid.nunique()

1235

In [6]:
df.columns

Index(['Unnamed: 0', 'companyid', 'keydevid', 'transcriptid', 'headline',
       'mostimportantdateutc', 'mostimportanttimeutc', 'keydeveventtypeid',
       'keydeveventtypename', 'companyname', 'transcriptcollectiontypeid',
       'transcriptcollectiontypename', 'transcriptpresentationtypeid',
       'transcriptpresentationtypename', 'transcriptcreationdate_utc',
       'transcriptcreationtime_utc', 'audiolengthsec', 'transcriptid.1',
       'transcriptcomponentid', 'componentorder', 'transcriptcomponenttypeid',
       'transcriptcomponenttypename', 'transcriptpersonid',
       'transcriptpersonname', 'proid', 'companyofperson', 'speakertypeid',
       'speakertypename', 'componenttextpreview', 'word_count',
       'componenttext'],
      dtype='object')

In [2]:
import pandas as pd
# Read the CSV file as a Dask DataFrame
filename = './transcript/2021.csv'
chunksize = 10000
n = 0

for chunk in pd.read_csv(filename, chunksize=chunksize):
    n += 1
    desc = chunk.describe()
    # Here, you would aggregate the statistics from each chunk.
    # This step requires careful handling since statistics like mean and std
    # cannot be directly averaged across chunks without knowing the total counts.
    # For a simple demonstration, you could print each chunk's description:
    print(desc)

print(n)

        Unnamed: 0     companyid      keydevid  transcriptid  \
count  10000.00000  1.000000e+04  1.000000e+04  1.000000e+04   
mean    4999.50000  5.598615e+07  6.993637e+08  2.165776e+06   
std     2886.89568  1.285805e+08  1.153470e+06  3.922076e+02   
min        0.00000  2.476600e+04  6.928753e+08  2.165168e+06   
25%     2499.75000  1.876620e+05  6.990847e+08  2.165486e+06   
50%     4999.50000  4.101800e+05  6.998533e+08  2.165815e+06   
75%     7499.25000  1.104186e+07  7.001174e+08  2.166227e+06   
max     9999.00000  7.001135e+08  7.004939e+08  2.166399e+06   

       keydeveventtypeid  transcriptcollectiontypeid  \
count       10000.000000                 10000.00000   
mean           70.034700                     4.60700   
std            48.086288                     2.51149   
min            48.000000                     1.00000   
25%            48.000000                     2.00000   
50%            51.000000                     7.00000   
75%            52.000000       

In [45]:
aggregated_stats

{}

In [26]:
surprise_df.SUESCORE.mean()

np.float64(0.6602050206225073)

In [ ]:
sorted(surprise_df.ANNDATS.unique())

[Timestamp('1992-04-21 00:00:00'),
 Timestamp('1993-01-04 00:00:00'),
 Timestamp('1993-01-05 00:00:00'),
 Timestamp('1993-01-06 00:00:00'),
 Timestamp('1993-01-08 00:00:00'),
 Timestamp('1993-01-10 00:00:00'),
 Timestamp('1993-01-11 00:00:00'),
 Timestamp('1993-01-12 00:00:00'),
 Timestamp('1993-01-13 00:00:00'),
 Timestamp('1993-01-16 00:00:00'),
 Timestamp('1993-01-18 00:00:00'),
 Timestamp('1993-01-19 00:00:00'),
 Timestamp('1993-01-20 00:00:00'),
 Timestamp('1993-01-22 00:00:00'),
 Timestamp('1993-01-25 00:00:00'),
 Timestamp('1993-01-26 00:00:00'),
 Timestamp('1993-01-27 00:00:00'),
 Timestamp('1993-01-28 00:00:00'),
 Timestamp('1993-01-29 00:00:00'),
 Timestamp('1993-02-01 00:00:00'),
 Timestamp('1993-02-02 00:00:00'),
 Timestamp('1993-02-03 00:00:00'),
 Timestamp('1993-02-05 00:00:00'),
 Timestamp('1993-02-08 00:00:00'),
 Timestamp('1993-02-09 00:00:00'),
 Timestamp('1993-02-10 00:00:00'),
 Timestamp('1993-02-11 00:00:00'),
 Timestamp('1993-02-12 00:00:00'),
 Timestamp('1993-02-

In [60]:
lower_quantile = surprise_df['SURPMEAN'].quantile(0.01)
upper_quantile = surprise_df['SURPMEAN'].quantile(0.99)
surprise_df['SURPMEAN'].clip(lower=lower_quantile, upper=upper_quantile).mean()

np.float64(0.4878067278712157)

In [61]:
lower_quantile = surprise_df['SUESCORE'].quantile(0.01)
upper_quantile = surprise_df['SUESCORE'].quantile(0.99)

surprise_df['SUESCORE'].clip(lower=lower_quantile, upper=upper_quantile).median()

np.float64(0.756255)

In [59]:
surprise_df

,OFTIC,TICKER,MEASURE,ANNDATS,FISCALP,PYEAR,PMON,ACTUAL,SUESCORE,SURPMEAN,SURPSTDEV,USFIRM
676,WB,002U,b'EPS',2014-05-21,b'QTR',2014.0,3.0,-0.03,0.70711,-0.04500,0.02121,1.0
677,WB,002U,b'EPS',2014-08-14,b'QTR',2014.0,6.0,-0.03,-0.50000,-0.02875,0.00250,1.0
678,WB,002U,b'EPS',2014-11-13,b'QTR',2014.0,9.0,-0.01,-0.08445,-0.00860,0.01658,1.0
680,WB,002U,b'EPS',2015-03-10,b'QTR',2014.0,12.0,0.04,0.27459,0.03780,0.00801,1.0
681,WB,002U,b'EPS',2015-05-14,b'QTR',2015.0,3.0,0.01,0.20964,0.00740,0.01240,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...
582533,ZOM,ZOM,b'EPS',2019-08-08,b'QTR',2019.0,6.0,-0.02,NaN,-0.06000,NaN,1.0
582534,ZOM,ZOM,b'EPS',2019-11-12,b'QTR',2019.0,9.0,-0.03,NaN,-0.02000,NaN,1.0
582536,ZOM,ZOM,b'EPS',2020-02-26,b'QTR',2019.0,12.0,-0.03,NaN,-0.02000,NaN,1.0
582537,ZOM,ZOM,b'EPS',2020-08-10,b'QTR',2020.0,6.0,-0.02,NaN,-0.01000,NaN,1.0


In [58]:
surprise_df[(surprise_df.SUESCORE <= 0.10) & (surprise_df.SUESCORE >= -0.10)]

,OFTIC,TICKER,MEASURE,ANNDATS,FISCALP,PYEAR,PMON,ACTUAL,SUESCORE,SURPMEAN,SURPSTDEV,USFIRM
678,WB,002U,b'EPS',2014-11-13,b'QTR',2014.0,9.0,-0.01,-0.08445,-0.00860,0.01658,1.0
706,WB,002U,b'EPS',2020-05-19,b'QTR',2020.0,3.0,0.30,0.08906,0.29690,0.03481,1.0
5644,SNAP,01FQ,b'EPS',2020-07-21,b'QTR',2020.0,6.0,-0.09,-0.08587,-0.08785,0.02508,1.0
8063,AA,AA,b'EPS',1995-01-11,b'QTR',1994.0,12.0,0.75,0.07778,0.74071,0.11939,1.0
8074,AA,AA,b'EPS',1997-04-04,b'QTR',1997.0,3.0,0.93,-0.05526,0.93846,0.15313,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...
579167,YHOO,YHOO,b'EPS',2010-01-26,b'QTR',2009.0,12.0,0.11,-0.06354,0.11172,0.02713,1.0
579174,YHOO,YHOO,b'EPS',2011-07-19,b'QTR',2011.0,6.0,0.18,0.01811,0.17982,0.00986,1.0
579177,YHOO,YHOO,b'EPS',2012-01-24,b'QTR',2011.0,12.0,0.24,-0.04053,0.24073,0.01803,1.0
582267,ZNGA,ZNGA,b'EPS',2014-11-06,b'QTR',2014.0,9.0,-0.01,0.09441,-0.01065,0.00689,1.0


In [3]:
ibes_df = pd.read_sas("./data/ibes1.sas7bdat")
ibes_df.CUSIP = ibes_df.CUSIP.apply(lambda x: x.decode('utf-8') if isinstance(x, bytes) else x)

ibes_df

,OFTIC,TICKER,ANNDATS,PDICITY,CUSIP,CNAME,PENDS,VALUE,USFIRM
0,NaN,b'AAAF',2002-05-06,b'ANN',02343610,b'AMEDISYS INC',2001-12-31,0.630,1.0
1,NaN,b'AAAF',2002-05-06,b'QTR',02343610,b'AMEDISYS INC',2001-12-31,0.280,1.0
2,NaN,b'ABFO',2015-02-17,b'ANN',E0003D11,b'ABERTIS INFRAEST',2014-12-31,0.676,1.0
3,NaN,b'ABX1',2002-04-24,b'QTR',00756M40,b'ADVANCED SEMI EN',2002-03-31,-0.010,1.0
4,NaN,b'ABXF',1986-03-14,b'ANN',02451E10,b'AMERICAN BARRICK',1985-12-31,0.210,1.0
...,...,...,...,...,...,...,...,...,...
965023,b'ZZZZ',b'CNSO',1998-08-18,b'QTR',20846210,b'CONSCO ENTERPRIS',1997-03-31,0.270,1.0
965024,b'ZZZZ',b'CNSO',1998-08-18,b'QTR',20846210,b'CONSCO ENTERPRIS',1997-06-30,0.170,1.0
965025,b'ZZZZ',b'CNSO',1998-08-18,b'QTR',20846210,b'CONSCO ENTERPRIS',1997-09-30,0.140,1.0
965026,b'ZZZZ',b'CNSO',1998-08-18,b'QTR',20846210,b'CONSCO ENTERPRIS',1997-12-31,0.170,1.0
